# Review Notes on Basic Automata

Feb 2025

## Deterministic Finite State Automaton

A finite state automaton is an abstraction that consists of a set of states, a set of allowable inputs, a transition function, an initial state, and an accepted state.

**Formal definition**: A finite state automaton consists of a tuple
$$
M=(Q, \Sigma, \delta, q_o, F)
$$
Where $Q$ is a set of finite states, $\Sigma$ is the set of allowable inputs, $\delta$ is the transition function that maps that inputs to state changes, $q_o$ is the initial state, and $F$ is the accepted state.

## Turnstyle FSA: 

A turnstyle is a simple example of a FSA. The turnstyle has two states $[locked, unlocked]$, two inputs $[Coin, Push]$. The initial state is locked. There is no accepted state, I'll just use an empty set. 

Let's say the turnstyle requires one quarter to unlock. 

I'll make a general FSA class below. 

In [17]:
class FSA:
    def __init__(self, states, inputs, transition_function, initial_state, final_state):
        self.states=states
        self.inputs=inputs
        self.transition_function= transition_function
        self.current_state= initial_state
        self.final_state= final_state

    def transition(self, input_symbol): 
        if input_symbol in self.inputs:
            if (self.current_state, input_symbol) in self.transition_function:
                self.current_state= self.transition_function[(self.current_state, input_symbol)]
                print(f' Transitioned to state: {self.current_state}')
            else:
                print(f' No transition defined for {self.current_state}, {input_symbol}')

        else: 
            print(f' Invalid Input: {input_symbol} must be in {self.states}') 

    def check_accepting(self):
        return self.current_state in self.final_state

    def get_state(self):
        return self.current_state

    

In [18]:
# Now to simulate the turnstyle
# Define the elements of the tuple; Using dictionaries

states= {'locked', 'unlocked'}
inputs= ['coin', 'push']
transition_function= {('locked', 'coin'): 'unlocked', ('unlocked', 'push'): 'locked'}
initial_state= 'locked'
final_state= set() # assume no accepting state

# Instantiate the model
turnstyle= FSA(states, inputs, transition_function, initial_state, final_state)

# Simulation
actions= ['push', 'coin', 'push', 'coin', 'push', 'coin', 'push']

# Print the outcomes
print('\nSimulating turnstyle FSA ... \n')
for action in actions:
    print(f' Action: {action}')
    turnstyle.transition(action)
    print(f' Current State: {turnstyle.get_state()}')
    print('-' * 30)


Simulating turnstyle FSA ... 

 Action: push
 No transition defined for locked, push
 Current State: locked
------------------------------
 Action: coin
 Transitioned to state: unlocked
 Current State: unlocked
------------------------------
 Action: push
 Transitioned to state: locked
 Current State: locked
------------------------------
 Action: coin
 Transitioned to state: unlocked
 Current State: unlocked
------------------------------
 Action: push
 Transitioned to state: locked
 Current State: locked
------------------------------
 Action: coin
 Transitioned to state: unlocked
 Current State: unlocked
------------------------------
 Action: push
 Transitioned to state: locked
 Current State: locked
------------------------------


## Push Down Automata
Extending the model to deal with the problem of inexact change to explore addition of simple stack memory. 

**Formal definition**: A pushdown automaton consists of a tuple with an additional term $\Gamma$ representing the stack symbols.

$$
M=(Q, \Sigma, \delta, \Gamma, q_o, F)
$$

Symbols can be stored in the stack, as determined by existing 'push' and 'pop' rules that modify the symbols in the stack. 

## Turnstyle PDA

The FSA turnstyle, could only take one coin that was the correct amount for unlocking. This is not very practical. To highlight the differences between these models, I'll add a stack memory mechanism to the turnstyle that stores dimes and nickels, with push/pop rules that compute the total amoung of change inserted (up to 25 cents). Just modifying the FSA for this...

In [36]:
# PDA Simulation Class

class PDA:
    def __init__(self, states, inputs, stack_symbols, transition_function, initial_state, final_state):
        self.states=states
        self.inputs=inputs
        self.stack_symbols=stack_symbols
        self.stack= [] # initialize list to hold the stack
        self.transition_function= transition_function
        self.current_state= initial_state
        self.final_state= final_state

    def stack_value(self):
        '''calcultes the value of coins in the stack'''
        value_map={'N':5, 'D':10, 'Q':25}
        return sum(value_map[coin] for coin in self.stack if coin in value_map)

    def transition(self, input_symbol):
        if input_symbol not in self.inputs:
            print(f'invalid input: {input_symbol}')
            return
            
        if (self.current_state, input_symbol) in self.transition_function:
            action= self.transition_function[(self.current_state, input_symbol)]
            # Stack operations
            if action.get('push'):
                self.stack.append(action['push']) # push coin to stack
            if action.get('pop') and self.stack:
                self.stack.pop() # pop from stack

            if self.stack_value() >=25:
                self.current_state= 'unlocked'
                print(f' Turnstyle Unlocked')
            else:
                self.current_state= action['next_state']
            
            # Clear stack if turnstyle is pushed
            if action.get('clear_stack'):
                self.stack.clear()
            
            # Print the outcome
            print(f' Action: {input_symbol}, New State: {self.current_state}, Stack: {self.stack}, Total: {self.stack_value()} cents')
        else:
            # Print statement for unacceptable input
            print(f' Action: {input_symbol}, No transition defined for ({self.current_state}, {input_symbol}).')


In [37]:
# Now to simulate the turnstyle as a PDA

states= {'locked', 'unlocked'}
inputs= {'nickel', 'dime', 'quarter', 'push'}
stack_symbols= {'N', 'D', 'Q'} 
Initial_state= 'locked'
final_state= set()

# Transition function
transition_function= {
    ('locked', 'nickel'): {'next_state': 'locked', 'push': 'N'},
    ('locked', 'dime'): {'next_state': 'locked', 'push': 'D'},
    ('locked', 'quarter'): {'next_state': 'locked', 'push': 'Q'},
    ('unlocked', 'push'): {'next_state': 'locked', 'clear_stack': True},
}

# Instantiate the model
Turnstyle_pda= PDA(states, inputs, stack_symbols, transition_function, initial_state, final_state)

# Simulate
#actions= ['nickel', 'nickel', 'nickel', 'nickel', 'nickel', 'push']
actions= ['nickel', 'nickel', 'dime', 'push', 'nickel', 'push']

print(f' Simulating')
for action in actions:
    result= Turnstyle_pda.transition(action)
    print("-" * 70)

 Simulating
 Action: nickel, New State: locked, Stack: ['N'], Total: 5 cents
----------------------------------------------------------------------
 Action: nickel, New State: locked, Stack: ['N', 'N'], Total: 10 cents
----------------------------------------------------------------------
 Action: dime, New State: locked, Stack: ['N', 'N', 'D'], Total: 20 cents
----------------------------------------------------------------------
 Action: push, No transition defined for (locked, push).
----------------------------------------------------------------------
 Turnstyle Unlocked
 Action: nickel, New State: unlocked, Stack: ['N', 'N', 'D', 'N'], Total: 25 cents
----------------------------------------------------------------------
 Turnstyle Unlocked
 Action: push, New State: unlocked, Stack: [], Total: 0 cents
----------------------------------------------------------------------


## Some notes of reflection on this PDA turnstyle machine...

* The stack memory models a physical mechanism which must allow coins to accumulate and be somehow sensitive to the weight. 
* The transition function is defined with the operation of the machine in mind. Where would a 'natural' transition function come from?
* The machine is self sustaining in that it will cycle through its states if inputs are available, and will not reach a final accepted state. 
* The state transitions could be represented as a random variable and should be proportional to the multiplicity? I wouldn't expect this to be uniform because there is only one transition that unlocks and several that result in a locked state. 
* This model describes the state transitions, but does not describe its dynamics. Import to note what this model does and does not account for. What is the connection, if any, between models of computation and system dynamics? Do these have to be analyzed separately?
* The machine is deterministic,i.e., no noise in its state transitions. 
* Each part in the machine has degrees of freedom. The turnstyle can rotate, etc. It is the interactions among the coupled sub-machines that constrain the sample space of the mechanism's overall degrees of freedom. 
* The above feature is important for the 'purposiveness' of the machine. If every part could access its full degrees of freedom, then the turnstyle would not function the way we need it to for access restriction. 

## Nested Stack Automaton (NSA)

I won't make another class here, because I think it would be a bit tedious to write, but I could update the current PDA turnstyle to a `nested stack automaton` by adding a second stack that handles euro currency. This would require the turnstyle to have a duplication mechanism for the USD stack that would come with a new transition function.

The PDA turnstyle is a fixed structure, and explores only the degrees of freedom that it already contains. The PDA turnstyle is a `self regenerating` process. As long as the structure remains in tact, it will maintain the ability to visit all its states given it sees the full range of inputs. From an energetic perspecitve it also constitutes a `work cycle` wherein the mechanism couples one or more energy dissipating processes with an energy input that resets the potential energy in the system. An `analagoous` system would be a Rube-Goldberg Machine that runs and must them be reset by the user. 

**New Problem**: The turnstyle solves the problem of access control. It should be able to ensure that people passing through have paid the appropriate amount of money and prevent them from passing otherwise. We could model the turnstyle interacting with this problem using a Poisson distributed queing problem. However, in order to do this, we need to make assumptions (or collect data) about the turnstyle's dynamics. `Is there a framework for incorporating dynamics into models of computation?`. This reminds me a bit of modeling metabolic systems. The mathematics that describe thermodynamics and kinetics are not connected, and must be separately modeled by collecting different types of observations about the system. 

**Relating to Biological Systems**: Biological systems consist of self-regenerating processes (e.g., metabolic work cycles). The key feature of biological systems is that they can duplicate these processes. As an example, the `genome` is part of an `autocatalytic set` that uses template replication to make copies of the entire set. The genome encodes proteins. The transcriptional/translational system makes copies of the proteins. The proteins have fixed functional degrees of freedom dictated by their structure (like tiny turnstyles). The ability of the system to solve problems is proportional to: 

1. A protein's degrees of freedom
2. The number of active copies of the protein

Variation in replication and regeneration add degrees of freedom to the functional state of the system, however, biological systems can't do this in a targeted way, it is only statistical. **Note**: many proteins can have their functional degrees of freedom modified by post-translational modification - the addition of covalently bonded chemical funtional groups. 

**A general informal conclusion**: I think there are two types of stack memory relevant to biological systems - stacks that store the current states of individual machines (stored within the structure of the machines) and stacks that store copies of the machines (stored within the collective structure of the system). The transition function for each machine modifies its available degrees of freedom (behavior). The transition function for the copy stack modifies the statistics of the total system. `Can nonlinear dynamics arise from feedback defined in the transition functions`. 


## A NestedStack Turnstile Model

Let:
- $q(t)$ be the queue size at discrete time $t$.
- $n(t)$ be the number of active turnstiles at time $t$.
- $k, m$ be threshold parameters.
- $\tau$ (decay_time) be the time (in seconds) after which an unused turnstile is removed.
- $u_i(t)$ be the last usage time of turnstile $i$.

1. **Queue Dynamics**  
   - When a person arrives:
     $$
     q(t+1) = q(t) + 1.
     $$
   - When a person passes (if $q(t) > 0$):
     $$
     q(t+1) = q(t) - 1.
     $$

2. **Turnstile Addition**  
   - If there are no active turnstiles ($n(t) = 0$) or the queue size exceeds $k \times n(t)$:
     $$
     \text{if } \bigl(n(t) = 0 \bigr) \;\text{or}\; \bigl(q(t+1) > k \, n(t)\bigr), \quad n(t+1) = n(t) + 1.
     $$

3. **Turnstile Removal**  
   - If there is more than one turnstile ($n(t) > 1$) and the queue size is below $m \times n(t)$:
     $$
     \text{if } \bigl(n(t) > 1\bigr) \;\text{and}\; \bigl(q(t+1) < m \, n(t)\bigr), \quad n(t+1) = n(t) - 1.
     $$

4. **Decay Condition**  
   - Each turnstile $i$ has a recorded last usage time $u_i(t)$. Let $T$ be the current clock time. If
     $$
     T - u_i(t) > \tau,
     $$
     then turnstile $i$ is removed.

Thus, the system adaptively adds turnstiles when $q(t)$ is large, removes turnstiles when $q(t)$ is small, and eliminates turnstiles that remain idle longer than $\tau$.


In [1]:
import time
import pandas as pd

class NestedStackTurnstile:
    def __init__(self, k=5, m=2, decay_time=10):
        """
        k: Threshold factor for adding a turnstile (queue size > k * active turnstiles)
        m: Threshold factor for removing a turnstile (queue size < m * active turnstiles)
        decay_time: Time (in seconds) before an idle turnstile is removed
        """
        self.queue_size = 0  # Number of people in the queue
        self.turnstile_stack = []  # Stack representing active turnstiles
        self.k = k
        self.m = m
        self.decay_time = decay_time
        self.turnstile_usage = {}  # Tracks last use of each turnstile

    def add_person(self):
        """A person arrives and joins the queue."""
        self.queue_size += 1
        self.check_turnstiles()

    def person_passes(self):
        """A person goes through a turnstile and leaves the queue."""
        if self.queue_size > 0:
            self.queue_size -= 1
            if self.turnstile_stack:
                turnstile = self.turnstile_stack[-1]  # Last turnstile used
                self.turnstile_usage[turnstile] = time.time()  # Update usage time

        self.check_turnstiles()

    def check_turnstiles(self):
        """Adjust the number of turnstiles based on queue size."""
        num_turnstiles = len(self.turnstile_stack)

        # Add turnstile if queue size exceeds k * number of turnstiles
        if num_turnstiles == 0 or self.queue_size > self.k * num_turnstiles:
            new_turnstile = f"T{num_turnstiles + 1}"
            self.turnstile_stack.append(new_turnstile)
            self.turnstile_usage[new_turnstile] = time.time()  # Mark turnstile as used

        # Remove turnstile if queue size is too small compared to active turnstiles
        if num_turnstiles > 1 and self.queue_size < self.m * num_turnstiles:
            removed_turnstile = self.turnstile_stack.pop()
            del self.turnstile_usage[removed_turnstile]

        # Check for turnstile decay
        self.remove_idle_turnstiles()

    def remove_idle_turnstiles(self):
        """Remove turnstiles that have been unused for longer than decay_time."""
        current_time = time.time()
        for turnstile in list(self.turnstile_stack):
            if current_time - self.turnstile_usage[turnstile] > self.decay_time:
                self.turnstile_stack.remove(turnstile)
                del self.turnstile_usage[turnstile]

    def get_status(self):
        """Returns the current status of the queue and turnstiles."""
        return {
            "queue_size": self.queue_size,
            "active_turnstiles": len(self.turnstile_stack),
            "turnstiles": list(self.turnstile_stack),
            "turnstile_usage": {t: time.time() - u for t, u in self.turnstile_usage.items()}
        }


# **Simulating the Nested Stack Turnstile System**
simulation = NestedStackTurnstile(k=3, m=1, decay_time=10)

# Randomized sequence of arrivals and departures
actions = [
    ("arrive", 10),  # 10 people arrive
    ("pass", 5),  # 5 people pass through
    ("arrive", 7),  # 7 more arrive
    ("pass", 10),  # 10 people pass (more than in queue)
    ("arrive", 3),  # 3 more arrive
    ("pass", 2),  # 2 people pass
]

# Running the simulation
simulation_results = []
for action, count in actions:
    for _ in range(count):
        if action == "arrive":
            simulation.add_person()
        elif action == "pass":
            simulation.person_passes()

        simulation_results.append(simulation.get_status())

# Convert to DataFrame for easy viewing
df = pd.DataFrame(simulation_results)
print(df)


    queue_size  active_turnstiles        turnstiles  \
0            1                  1              [T1]   
1            2                  1              [T1]   
2            3                  1              [T1]   
3            4                  2          [T1, T2]   
4            5                  2          [T1, T2]   
5            6                  2          [T1, T2]   
6            7                  3      [T1, T2, T3]   
7            8                  3      [T1, T2, T3]   
8            9                  3      [T1, T2, T3]   
9           10                  4  [T1, T2, T3, T4]   
10           9                  4  [T1, T2, T3, T4]   
11           8                  4  [T1, T2, T3, T4]   
12           7                  4  [T1, T2, T3, T4]   
13           6                  4  [T1, T2, T3, T4]   
14           5                  4  [T1, T2, T3, T4]   
15           6                  4  [T1, T2, T3, T4]   
16           7                  4  [T1, T2, T3, T4]   
17        

In [2]:
import time
import pandas as pd
import random

class NestedStackTurnstile:
    def __init__(self, k=5, m=2, decay_time=10):
        """
        k: Threshold factor for adding a turnstile (queue size > k * active turnstiles)
        m: Threshold factor for removing a turnstile (queue size < m * active turnstiles)
        decay_time: Time (in seconds) before an idle turnstile is removed
        """
        self.queue_size = 0  # Number of people in the queue
        self.turnstile_stack = []  # Stack representing active turnstiles
        self.k = k
        self.m = m
        self.decay_time = decay_time
        self.turnstile_usage = {}  # Tracks last use of each turnstile

    def add_person(self):
        """A person arrives and joins the queue."""
        self.queue_size += 1
        self.check_turnstiles()

    def person_passes(self):
        """
        A person goes through one of the active turnstiles (chosen among all available).
        This allows concurrency: multiple turnstiles can be used in parallel
        rather than always using the 'last' turnstile.
        """
        if self.queue_size > 0:
            self.queue_size -= 1
            if self.turnstile_stack:
                # Pick any turnstile (example: random choice)
                turnstile = random.choice(self.turnstile_stack)
                self.turnstile_usage[turnstile] = time.time()  # Update usage time
        self.check_turnstiles()

    def check_turnstiles(self):
        """Adjust the number of turnstiles based on queue size."""
        num_turnstiles = len(self.turnstile_stack)

        # Add turnstile if queue size exceeds k * number of turnstiles
        if num_turnstiles == 0 or self.queue_size > self.k * num_turnstiles:
            new_turnstile = f"T{num_turnstiles + 1}"
            self.turnstile_stack.append(new_turnstile)
            self.turnstile_usage[new_turnstile] = time.time()

        # Remove turnstile if queue size is too small compared to active turnstiles
        num_turnstiles = len(self.turnstile_stack)  # re-check after a possible addition
        if num_turnstiles > 1 and self.queue_size < self.m * num_turnstiles:
            removed_turnstile = self.turnstile_stack.pop()
            del self.turnstile_usage[removed_turnstile]

        # Check for turnstile decay
        self.remove_idle_turnstiles()

    def remove_idle_turnstiles(self):
        """Remove turnstiles that have been unused for longer than decay_time."""
        current_time = time.time()
        for turnstile in list(self.turnstile_stack):
            if current_time - self.turnstile_usage[turnstile] > self.decay_time:
                self.turnstile_stack.remove(turnstile)
                del self.turnstile_usage[turnstile]

    def get_status(self):
        """Returns the current status of the queue and turnstiles."""
        return {
            "queue_size": self.queue_size,
            "active_turnstiles": len(self.turnstile_stack),
            "turnstiles": list(self.turnstile_stack),
            "turnstile_usage": {t: time.time() - u for t, u in self.turnstile_usage.items()}
        }

# Example simulation
if __name__ == "__main__":
    simulation = NestedStackTurnstile(k=3, m=1, decay_time=10)

    # Randomized sequence of arrivals and departures
    actions = [
        ("arrive", 10),  # 10 people arrive
        ("pass", 5),     # 5 people pass
        ("arrive", 7),   # 7 more arrive
        ("pass", 10),    # 10 people pass
        ("arrive", 3),   # 3 more arrive
        ("pass", 2),     # 2 people pass
    ]

    # Running the simulation
    simulation_results = []
    for action, count in actions:
        for _ in range(count):
            if action == "arrive":
                simulation.add_person()
            elif action == "pass":
                simulation.person_passes()
            # Capture state after each event
            simulation_results.append(simulation.get_status())

    # Convert results to DataFrame
    df = pd.DataFrame(simulation_results)
    print(df)


    queue_size  active_turnstiles        turnstiles  \
0            1                  1              [T1]   
1            2                  1              [T1]   
2            3                  1              [T1]   
3            4                  2          [T1, T2]   
4            5                  2          [T1, T2]   
5            6                  2          [T1, T2]   
6            7                  3      [T1, T2, T3]   
7            8                  3      [T1, T2, T3]   
8            9                  3      [T1, T2, T3]   
9           10                  4  [T1, T2, T3, T4]   
10           9                  4  [T1, T2, T3, T4]   
11           8                  4  [T1, T2, T3, T4]   
12           7                  4  [T1, T2, T3, T4]   
13           6                  4  [T1, T2, T3, T4]   
14           5                  4  [T1, T2, T3, T4]   
15           6                  4  [T1, T2, T3, T4]   
16           7                  4  [T1, T2, T3, T4]   
17        

In [3]:
import time
import pandas as pd
import numpy as np
import random

class NestedStackTurnstile:
    def __init__(self, k=3, m=1, decay_time=10):
        """
        k: Threshold factor for adding a turnstile (queue size > k * active turnstiles)
        m: Threshold factor for removing a turnstile (queue size < m * active turnstiles)
        decay_time: Time (in seconds) before an idle turnstile is removed
        """
        self.queue_size = 0  # Number of people in the queue
        self.turnstile_stack = []  # List representing active turnstiles
        self.k = k
        self.m = m
        self.decay_time = decay_time
        self.turnstile_usage = {}  # Tracks last use (timestamp) of each turnstile

    def add_person(self, event_time=None):
        """A person arrives and joins the queue."""
        self.queue_size += 1
        self.check_turnstiles()

    def person_passes(self):
        """
        One person passes through one of the active turnstiles.
        Now we choose a random turnstile (parallel usage).
        """
        if self.queue_size > 0:
            self.queue_size -= 1
            if self.turnstile_stack:
                turnstile = random.choice(self.turnstile_stack)
                self.turnstile_usage[turnstile] = time.time()  # Mark as used
        self.check_turnstiles()

    def check_turnstiles(self):
        """Adjust the number of turnstiles based on queue size."""
        num_turnstiles = len(self.turnstile_stack)

        # 1) Possibly add a new turnstile
        if num_turnstiles == 0 or self.queue_size > self.k * num_turnstiles:
            new_turnstile = f"T{num_turnstiles + 1}"
            self.turnstile_stack.append(new_turnstile)
            self.turnstile_usage[new_turnstile] = time.time()

        # 2) Possibly remove one turnstile (never removing the last one)
        num_turnstiles = len(self.turnstile_stack)
        if num_turnstiles > 1 and self.queue_size < self.m * num_turnstiles:
            removed_turnstile = self.turnstile_stack.pop()
            del self.turnstile_usage[removed_turnstile]

        # 3) Remove turnstiles if they've been idle longer than decay_time
        self.remove_idle_turnstiles()

    def remove_idle_turnstiles(self):
        """Remove turnstiles unused for longer than self.decay_time seconds."""
        current_time = time.time()
        for turnstile in list(self.turnstile_stack):
            if current_time - self.turnstile_usage[turnstile] > self.decay_time:
                self.turnstile_stack.remove(turnstile)
                del self.turnstile_usage[turnstile]

    def get_status(self, sim_clock):
        """Current status of the queue and turnstiles, with a simulation clock."""
        return {
            "time": sim_clock,
            "queue_size": self.queue_size,
            "active_turnstiles": len(self.turnstile_stack),
            "turnstiles": list(self.turnstile_stack),
        }

def simulate_poisson(
    arrival_rate=1.0,
    service_rate=0.5,
    sim_time=50.0,
    k=3,
    m=1,
    decay_time=10,
    seed=None
):
    """
    Simulates a queue with:
      - Poisson arrivals at rate = arrival_rate
      - Service times ~ Exponential( service_rate * number_of_turnstiles ), 
        i.e. total service rate = service_rate * n(t).

    The simulation runs until the clock reaches sim_time.
    """
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)
    
    # Instantiate the NestedStackTurnstile
    turnstile_system = NestedStackTurnstile(k=k, m=m, decay_time=decay_time)
    
    # Simulation clock
    t = 0.0

    # Schedule first arrival
    # Inter-arrival time ~ Exp(arrival_rate)
    t_arrival = np.random.exponential(1.0 / arrival_rate)
    
    # No departures scheduled until we have at least 1 person in queue
    t_departure = float('inf')

    # For collecting data
    records = []
    
    # Run until sim_time
    while t < sim_time:
        # Pick the next event
        next_event_time = min(t_arrival, t_departure)
        
        if next_event_time > sim_time:
            # We won't process an event that occurs after sim_time
            break
        
        # Advance clock
        t = next_event_time
        
        # Check if the next event is an arrival or a departure
        if t == t_arrival:
            # Arrival event
            turnstile_system.add_person(event_time=t)
            
            # Schedule the next arrival
            t_arrival = t + np.random.exponential(1.0 / arrival_rate)
            
            # If the queue was empty before, we may need to schedule departure
            # If queue_size is now 1 (and there's at least 1 turnstile),
            # we can schedule a new departure
            if turnstile_system.queue_size == 1 and len(turnstile_system.turnstile_stack) > 0:
                rate = service_rate * len(turnstile_system.turnstile_stack)
                t_departure = t + np.random.exponential(1.0 / rate)
        
        else:
            # Departure event
            turnstile_system.person_passes()
            
            # If the queue is still > 0, schedule the next departure
            # based on current # of turnstiles
            if turnstile_system.queue_size > 0 and len(turnstile_system.turnstile_stack) > 0:
                rate = service_rate * len(turnstile_system.turnstile_stack)
                t_departure = t + np.random.exponential(1.0 / rate)
            else:
                t_departure = float('inf')
        
        # Record system status after each event
        records.append(turnstile_system.get_status(sim_clock=t))
    
    # Convert to DataFrame
    df = pd.DataFrame(records)
    return df

if __name__ == "__main__":
    # Example usage
    df_results = simulate_poisson(
        arrival_rate=1.0,     # lambda
        service_rate=0.5,     # mu for EACH turnstile
        sim_time=50.0,        # total simulation time
        k=2,                  # threshold for adding turnstile
        m=1,                  # threshold for removing turnstile
        decay_time=15,        # turnstile removed if idle > 15s
        seed=42               # reproducible randomization
    )
    
    print(df_results)


         time  queue_size  active_turnstiles    turnstiles
0    0.469268           1                  1          [T1]
1    3.102759           0                  1          [T1]
2    3.479390           1                  1          [T1]
3    3.818639           0                  1          [T1]
4    4.392332           1                  1          [T1]
..        ...         ...                ...           ...
83  48.051943           6                  3  [T1, T2, T3]
84  48.260208           5                  3  [T1, T2, T3]
85  48.445257           6                  3  [T1, T2, T3]
86  49.132126           5                  3  [T1, T2, T3]
87  49.460146           6                  3  [T1, T2, T3]

[88 rows x 4 columns]


CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 11.7 µs


()